In [1]:
import numpy as np
import tensorflow as tf

In [2]:
print("Tensorflow Version: {}".format(tf.version.VERSION))
print("NumPy Version:      {}".format(np.version.version))

Tensorflow Version: 2.1.0
NumPy Version:      1.18.1
